# Credit Risk Classification - Challenge

In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
df_lending_data = pd.read_csv(Path('Resources/lending_data.csv'))

# Review the DataFrame
df_lending_data.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = df_lending_data["loan_status"]

# Separate the X variable, the features
X = df_lending_data.drop(columns="loan_status")

In [4]:
# Review the y variable Series
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [5]:
# Review the X variable DataFrame
X[:5]

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [6]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [7]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [8]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using training data
classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [9]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [10]:
# Print the balanced_accuracy score of the model
print(f'Balanced accuracy score of the model: {balanced_accuracy_score(y_test, predictions)}')

Balanced accuracy score of the model: 0.9442676901753825


In [11]:
# Generate a confusion matrix for the model
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(cm, index=["Actual - Healthy Loan (0)", "Actual - High-Risk Loan (1)"], columns=["Predicted - Healthy Loan (0)", "Predicted - High-Risk Loan (1)"])
cm_df

,Predicted - Healthy Loan (0),Predicted - High-Risk Loan (1)
Actual - Healthy Loan (0),18679,80
Actual - High-Risk Loan (1),67,558


In [12]:
# Print the classification report for the model
target_names = ["Healthy Loan (0)", "High-Risk Loan (1)"]
print(classification_report(y_test, predictions, target_names=target_names))

                    precision    recall  f1-score   support

  Healthy Loan (0)       1.00      1.00      1.00     18759
High-Risk Loan (1)       0.87      0.89      0.88       625

          accuracy                           0.99     19384
         macro avg       0.94      0.94      0.94     19384
      weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:**

* Precision: For the "Healthy Loan" class (label 0), the precision is 1.00, which means that all the instances predicted as "Healthy Loan" are indeed true. For the "High-Risk Loan" class (label 1), the precision is 0.87, indicating that 87% of the instances predicted as "High-Risk Loan" are true positives.

* Recall: For the "Healthy Loan" class (label 0), the recall is 1.00, meaning that the model correctly identifies all instances of "Healthy Loan." For the "High-Risk Loan" class (label 1), the recall is 0.89, indicating that the model captures 89% of the actual "High-Risk Loan" instances.

* Accuracy: Accuracy measures the overall correctness of the model's predictions. In this case, the accuracy is 0.99, which means that the model correctly predicts the label for 99% of the instances in the dataset.

Overall, the logistic regression model appears to perform very well in predicting both the "Healthy Loan" (label 0) and "High-Risk Loan" (label 1) classes. It achieves a high precision, recall, and F1-score for the "Healthy Loan" class, indicating accurate predictions and capture of all instances. For the "High-Risk Loan" class, the model has a slightly lower precision, recall, and F1-score but still performs well, capturing a high percentage of actual high-risk loans while maintaining a good balance between precision and recall. The overall accuracy of the model is 99%, indicating a high level of correctness in its predictions.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [13]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
ros_model = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_train_ros, y_train_ros= ros_model.fit_resample(X_train, y_train)

In [19]:
# Count the distinct values of the resampled labels data
y_train_ros.value_counts()

1    56277
0    56277
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [21]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier_ros = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
classifier_ros.fit(X_train_ros, y_train_ros)

# Make a prediction using the testing data
ros_predictions = classifier_ros.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [22]:
# Print the balanced_accuracy score of the model 
print(f'Balanced accuracy score of the model ROS: {balanced_accuracy_score(y_test, ros_predictions)}')

Balanced accuracy score of the model ROS: 0.9959744975744975


In [23]:
# Generate a confusion matrix for the model
cm_ros = confusion_matrix(y_test, ros_predictions)
cm_ros_df = pd.DataFrame(cm_ros, index=["Actual - Healthy Loan (0)", "Actual - High-Risk Loan (1)"], columns=["Predicted - Healthy Loan (0)", "Predicted - High-Risk Loan (1)"])
cm_ros_df

,Predicted - Healthy Loan (0),Predicted - High-Risk Loan (1)
Actual - Healthy Loan (0),18668,91
Actual - High-Risk Loan (1),2,623


In [24]:
# Print the classification report for the model
target_names_ros = ["Healthy Loan (0)", "High-Risk Loan (1)"]
print(classification_report(y_test, ros_predictions, target_names=target_names_ros))

                    precision    recall  f1-score   support

  Healthy Loan (0)       1.00      1.00      1.00     18759
High-Risk Loan (1)       0.87      1.00      0.93       625

          accuracy                           1.00     19384
         macro avg       0.94      1.00      0.96     19384
      weighted avg       1.00      1.00      1.00     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:**

* Precision: For the "Healthy Loan" class (0), the precision is 1.00, indicating that all the predicted healthy loans are correct. For the "High-Risk Loan" class (1), the precision is 0.87, suggesting that 87% of the predicted high-risk loans are correct.

* Recall: For the "Healthy Loan" class (0), the recall is 1.00, indicating that all the healthy loans are correctly identified. For the "High-Risk Loan" class (1), the recall is 1.00, meaning that all the high-risk loans are correctly identified.

* Accuracy: Accuracy is the overall proportion of correctly classified instances. In this case, the accuracy is 1.00, meaning that the logistic regression model accurately predicts both the "Healthy Loan" and "High-Risk Loan" labels.

Overall, the logistic regression model demonstrates excellent performance in predicting both the "Healthy Loan" (0) and "High-Risk Loan" (1) labels. It achieves high precision, recall, and F1-score for both classes, indicating that it effectively distinguishes between healthy and high-risk loans. The overall accuracy of 1.00 further confirms the model's capability to make accurate predictions.